In [2]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData

### Hyper parameters setting

In [2]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cpu',
    'attrs': ['title'],
    'k': 0,
    'validate':False
}
# torch.cuda.set_device(hparams['device'])

### Construct necessary dictionaries
- already done and the results are in `data/dictionaries`

In [ ]:
# constructBasicDict(attrs=['title'],path='/home/peitian_zhang/Data/MIND')

In [5]:
from utils.utils import constructNid2idx,constructUid2idx

In [6]:
constructUid2idx(['/home/peitian_zhang/Data/MIND/MINDlarge_whole/behaviors.tsv'],'whole')

### View data

In [3]:
vocab, loaders = prepare(hparams, pin_memory=False)

# for encoding news only
# vocab, loaders = prepare(hparams, news=True)

[2021-03-24 10:24:31,216] INFO (root) Hyper Parameters are
{'npratio': 4, 'mode': 'train', 'scale': 'demo', 'batch_size': 10, 'his_size': 50, 'title_size': 15, 'device': 'cpu', 'attrs': ['title'], 'k': 0, 'validate': False}
[2021-03-24 10:24:31,217] INFO (root) preparing dataset...
[2021-03-24 10:24:33,455] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
# loader_train
a = next(iter(loaders[0]))
# loader_dev
b = next(iter(loaders[1]))

### Tailor Data to demo size

In [ ]:
# tailor 2000 impressions from MINDsmall_train to form MINDdemo_train
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [4]:
analyse(hparams)

avg_title_length:10.67731736385395
 avg_abstract_length:36.4448570331045
 avg_his_length:32.99787212887438
 avg_impr_length:37.40116394684935
 cnt_his_lg_50:447829
 cnt_his_eq_0:46065
 cnt_imp_multi:567571


## The rest is developing